In [11]:
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
#tf.enable_eager_execution()
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np
import pickle
import numpy as np

In [12]:
with open('encodedStates.pickle', 'rb') as handle:
    data = pickle.load(handle)

In [13]:
def df_to_np(dataframe,shuffle=True,batch_size=32,target="HHVEHCNT"):
    dataframe = dataframe.copy()
    labels = dataframe.pop(target)
    return dataframe.values, labels.values

In [14]:
class TransferLearn:
    
    def __init__(self, fromStateData, toStateData, layers_to_transfer, verbose):
        self.data = data
        self.fromState = fromStateData
        self.toState = toStateData
        self.layers_to_transfer = layers_to_transfer
        self.verbose = verbose
        
        self.fromModelWeights = None
        
        self.train_X_from, self.train_y_from = df_to_np(fromStateData)
        self.train_X_to, self.train_y_to = df_to_np(toStateData)
    
    #Get the weights from model built on toStateData
    def getFromModelWeights(self, batch_size=100, validation_split=0.2, epochs=5):
        train_X_from = self.train_X_from.copy()
        train_y_from = self.train_y_from.copy()
        model = tf.keras.Sequential([
          tf.keras.layers.Dense(128,activation="relu"),
          tf.keras.layers.Dense(128),
          tf.keras.layers.Dense(128),
          tf.keras.layers.Dense(128),
          tf.keras.layers.Dense(64,activation="relu"),
          tf.keras.layers.Dense(4,activation="softmax")
        ])
        
        model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

        model.fit(x=train_X_from, y = train_y_from,epochs=5, batch_size = batch_size, validation_split=validation_split, shuffle=True, verbose=self.verbose)
        
        weights = []
        for i in range(0,self.layers_to_transfer):
            weights.append(model.layers[i].get_weights())
        return weights
    
    def transfer(self, trainable=True, batch_size=100, validation_split=0.2, epochs=5):
        train_X_to = self.train_X_to.copy()
        train_y_to = self.train_y_to.copy()
        weights = self.getFromModelWeights()
        print("*****weights obtained from fromStateData*****")
        
        print("Transferring")
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Dense(128,activation="relu", weights=weights[0], trainable=trainable))
        for i in range(1,self.layers_to_transfer):
            model.add(tf.keras.layers.Dense(128, weights=weights[i], trainable=trainable))
        for i in range(4-self.layers_to_transfer,4):
            model.add(tf.keras.layers.Dense(128))
        model.add(tf.keras.layers.Dense(64,activation="relu"))
        model.add(tf.keras.layers.Dense(4,activation="softmax"))

        
        model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        print("what the fuck", train_X_to.shape)
        history = model.fit(x=train_X_to, y = train_y_to, epochs=epochs, batch_size = batch_size, validation_split=validation_split, shuffle=True,verbose=self.verbose)
        val_acc = history.history["val_accuracy"][-1]
        print("*****Transferring done*****", "Trainable:", trainable)
        return model, val_acc
    
    
    def benchmark(self, batch_size=100, validation_split=0.2, epochs=5):
        train_X_to = self.train_X_to.copy()
        train_y_to = self.train_y_to.copy()
        print("*****Training Benchmark Model*****")
        model = tf.keras.Sequential([
          tf.keras.layers.Dense(128,activation="relu"),
          tf.keras.layers.Dense(128),
          tf.keras.layers.Dense(128),
          tf.keras.layers.Dense(128),
          tf.keras.layers.Dense(64,activation="relu"),
          tf.keras.layers.Dense(4,activation="softmax")
        ])
        
        model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

        history = model.fit(x=train_X_to, y = train_y_to,epochs=5, batch_size = batch_size, validation_split=validation_split, shuffle=True,verbose=self.verbose)
        val_acc = history.history["val_accuracy"][-1]
        print("*****Training Benchmark Model Done*****")
        return model, val_acc
    
    def compare(self):
        benchmark_model, benchmark_acc = self.benchmark()
        frozenTransfer_model, frozenTransfer_acc = self.transfer(trainable=False)
        unfrozenTransfer_model, unfrozenTransfer_acc = self.transfer(trainable=True)
        
        return (benchmark_acc-frozenTransfer_acc, benchmark_acc-unfrozenTransfer_acc)
        
    

In [19]:
MA_to_CA = TransferLearn(fromStateData = data["MA"], toStateData = data["NY"], layers_to_transfer=2, verbose = 1)
compareMAandCA = MA_to_CA.compare()

*****Training Benchmark Model*****
Train on 81690 samples, validate on 20423 samples
Epoch 1/5
81690/81690 [==============================] - 4s 50us/sample - loss: 0.9421 - accuracy: 0.5814 - val_loss: 0.9457 - val_accuracy: 0.5875
Epoch 2/5
81690/81690 [==============================] - 3s 37us/sample - loss: 0.9227 - accuracy: 0.5923 - val_loss: 0.9479 - val_accuracy: 0.5964
Epoch 3/5
81690/81690 [==============================] - 3s 35us/sample - loss: 0.9183 - accuracy: 0.5953 - val_loss: 0.9483 - val_accuracy: 0.5852
Epoch 4/5
81690/81690 [==============================] - 3s 35us/sample - loss: 0.9160 - accuracy: 0.5964 - val_loss: 0.9530 - val_accuracy: 0.5975
Epoch 5/5
81690/81690 [==============================] - 3s 35us/sample - loss: 0.9141 - accuracy: 0.5981 - val_loss: 0.9518 - val_accuracy: 0.5874
*****Training Benchmark Model Done*****
Train on 2568 samples, validate on 643 samples
Epoch 1/5
2568/2568 [==============================] - 1s 314us/sample - loss: 1.0717 - 

In [6]:
compareMAandCA

(0.00092953444, 0.014359415)

In [7]:
print(data.keys())

dict_keys(['NC', 'WI', 'NY', 'MD', 'PA', 'CA', 'TX', 'AZ', 'WA', 'IL', 'KY', 'MT', 'IA', 'GA', 'ME', 'VA', 'SC', 'WV', 'FL', 'NH', 'MN', 'NE', 'AR', 'NJ', 'SD', 'NM', 'OK', 'MI', 'VT', 'ID', 'DE', 'MA', 'WY', 'CO', 'IN', 'AL', 'TN', 'HI', 'AK', 'OH', 'RI', 'LA', 'OR', 'KS', 'UT', 'MO', 'DC', 'NV', 'ND', 'MS', 'CT'])


In [8]:
states = ["MA", "CA", "NY"]
I,J = len(states), len(states)
similarity_list = []

In [12]:
layers_to_transfer = 4
similarity_list.append(np.zeros((2,I,J)))
#We can only transfer at most 4 layers for now
for n in range(1,layers_to_transfer + 1):
    #n of IxJ matrices
    similarity = np.zeros((2,I,J))
    for r in range(I):
        fromState = states[r]
        for c in range(J):
            toState = states[c]
            print("Processing", fromState, toState)
            if fromState != toState:
                transferLearner = TransferLearn(fromStateData = data[fromState], toStateData = data[toState], layers_to_transfer=n,verbose=0)
                frozen_diff, unfrozen_diff = transferLearner.compare()
            else:
                frozen_diff, unfrozen_diff = 0, 0
            similarity[0][r][c] = frozen_diff
            similarity[1][r][c] = unfrozen_diff
    print(similarity)
    similarity_list.append(similarity)
    

Processing MA MA
Processing MA CA
*****Training Benchmark Model*****
*****Training Benchmark Model Done*****
*****weights obtained from fromStateData*****
Transferring
*****Transferring done***** Trainable: False
*****weights obtained from fromStateData*****
Transferring
*****Transferring done***** Trainable: True
Processing MA NY
*****Training Benchmark Model*****
*****Training Benchmark Model Done*****
*****weights obtained from fromStateData*****
Transferring


ValueError: Layer weight shape (15, 128) not compatible with provided weight shape (16, 128)